In [103]:
import os
import sys
import gc
import random
import datetime
import time

import random
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

seed = 3
random.seed(seed)
np.random.seed(seed)

In [262]:
sample_mall.time_stamp.max()

Timestamp('2017-09-14 22:40:00')

In [2]:
path = '/home/zhouzr/data/user-location-forcast/'
os.chdir(path)
train = pd.read_csv('./original_data/训练数据-ccf_first_round_user_shop_behavior.csv')
shop = pd.read_csv('./original_data/训练数据-ccf_first_round_shop_info.csv')
test = pd.read_csv('./original_data/AB榜测试集-evaluation_public.csv')

In [3]:
train = pd.merge(train, shop.drop(['longitude','latitude'],axis=1), on='shop_id', how='left')
sample = pd.concat([train, test],ignore_index=True)
sample['time_stamp'] = pd.to_datetime(sample.time_stamp)
sample.insert(0,'sample_id',range(sample.shape[0]))
mall_id_list = shop.mall_id.unique()
del train, test; gc.collect()

42

In [61]:
def gen_train_test(df, feats, test=True, xgbformat=False):
    if test:
        train_idx = df[df.time_stamp<='2017-8-24'].index.tolist()
        test_idx = df[(df.time_stamp>='2017-8-25')&(df.time_stamp<='2017-8-31')].index.tolist()
    else:
        train_idx = df[df.time_stamp<='2017-8-31'].index.tolist()
        test_idx = df[df.time_stamp>='2017-9-1'].index.tolist()
    
    if xgb_format:
        pass
    else:
        train_x = feats[train_idx]
        test_x = feats[test_idx]
        train_y = df.shop_id.values[train_idx]
        test_y = df.shop_id.values[test_idx]
        test_row_id = df.row_id.values[test_idx]
        return train_x, train_y, test_x, test_y, test_row_id

In [157]:
str(mall_id_list_sub)
result_record = {'test_mall':[],'drop_outlier':[],'features_desc':[],'test_score':[],'train_score':[]}

"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_3832']"

In [264]:
model_xgb

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [311]:
model_xgb = xgb.XGBClassifier(learning_rate=0.05, max_depth=5,n_estimators=300, reg_alpha=0.3, gamma=0.3, silent=False, subsample=0.7)
model_lr = LogisticRegression()
model_rf = RandomForestClassifier()
models = {#'xgb':model_xgb, 
          #'lr':model_lr, 
    'rf':model_xgb}


result = {'mall':[],'model_name':[], 'train_score':[], 'test_score':[], 'train_time':[], 'feat_time':[]}
for mall in mall_id_list_sub:
    t = time.time()
    sample_mall = sample[sample.mall_id == mall].copy().reset_index(drop=True)
    shop_mall = shop[shop.mall_id == mall].copy().reset_index(drop=True)
    
    features = []
    
    # outlier filter
    outlierFilter = OutlierFilter()
    sample_mall = outlierFilter.run(sample_mall)
    
    # wifi features
    wififeatExt = WifiFeatExt()
    wififeatExt.fit(sample_mall, drop_num=30)
#     features.append(wififeatExt.extract_wifi_connect(sample_mall))
    features.append(wififeatExt.extract_wifi_power(sample_mall))
    
    # time features
    timefeatExt = TimeFeatExt()
    features.append(timeFeat.extract_weekday_hour_map(sample_mall))
    
    # space features
    sapecefeatExt = SpaceFeatExt()
    features.append(sapecefeatExt.extract_space(sample_mall))
    

    # user_id 
    userfeatExt = UserFeatExt()
    features.append(userfeatExt.extract_user_id(sample_mall))
    
    
    features = np.concatenate(features, axis=1)
    train_x, train_y, test_x, test_y, _ = gen_train_test(sample_mall, features)
    t_feat_time = time.time() - t
    for model_name, model in models.items():
        t = time.time()
        if model_name == 'xgb':
            model.fit(train_x, train_y,
                     eval_set=[(train_x, train_y), (test_x, test_y)], eval_metric='auc', verbose=True)
        else:
            model.fit(train_x, train_y)
        t_model_time = time.time() - t
        result['mall'].append(mall)
        result['model_name'].append(model_name)
        result['train_score'].append(accuracy_score(model.predict(train_x), train_y))
        result['test_score'].append(accuracy_score(model.predict(test_x), test_y))
        result['feat_time'].append(t_feat_time)
        result['train_time'].append(t_model_time)

In [312]:
result = pd.DataFrame(result)
result_record['test_mall'].append(str(mall_id_list_sub))
result_record['drop_outlier'].append('yes')
result_record['features_desc'].append('wifi + time + space + user_id2 + xgb300_earlyshop++')
result_record['test_score'].append(result.test_score.mean())
result_record['train_score'].append(result.train_score.mean())

In [313]:
pd.DataFrame(result_record)

,drop_outlier,features_desc,test_mall,test_score,train_score
0,yes,wifi + space + time,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.872513,0.998100
1,no,wifi + space + time,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.869610,0.997817
2,no,wifi + time,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.862308,0.996574
3,yes,wifi + time,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.866606,0.997325
4,yes,wifi +space + time + space_dist,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.808989,0.994327
5,yes,wifi + time + space_dist,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.810424,0.994765
6,yes,wifi + time + space + user_id,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.809554,0.994192
7,yes,wifi + time + space + user_id,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.811697,0.994061
8,yes,wifi + time + space + user_id,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.872357,0.998538
9,yes,wifi + time + space + user_id2,"['m_4548', 'm_4515', 'm_5825', 'm_2878', 'm_38...",0.872763,0.998045


In [134]:
class OutlierFilter(object):
    
    def __init__(self):
        pass
    
    def run(self, df):
        outlier_idx1 = self.space_outlier_detect(df)
        outlier_idx2 = self.user_outlier_detect(df)
        test_idx = df[df.row_id.notnull()].index.tolist()
        outlier_idx = list(set(outlier_idx1 + outlier_idx2) - set(test_idx))
        result = df.drop(outlier_idx, axis=0).reset_index(drop=True)
        return result
    
    def space_outlier_detect(self, df):
        def outlier_filter(df, col, min_filter=0.005, max_filter=0.995):
            min_outlier = np.percentile(df[col], min_filter*100)
            max_outlier = np.percentile(df[col], max_filter*100)
            outlier_index = df[(df[col] > max_outlier) | (df[col] < min_outlier)].index.tolist()
            return outlier_index
        idx1 = outlier_filter(df, 'latitude')
        idx2 = outlier_filter(df, 'longitude')
        return list(set(idx1 + idx2))
    
    def user_outlier_detect(self, df):
        user_behavior = df.groupby('user_id')['sample_id'].count()
        idx = user_behavior[user_behavior>100].index.tolist()
        return idx

class UserFeatExt(object):
    
    def __init__(self):
        pass
    
    def extract_user_id(self, df_):
        df = df_.copy()
        user_cnt = df.groupby('user_id')['user_id'].count()
        user_set = set(df[df.shop_id.isnull()].user_id.values) |\
        set(df[df.shop_id.notnull()].user_id.values) |\
        set(user_cnt[user_cnt>5].index.tolist())
        
        df.loc[-df.user_id.isin(user_set),'user_id'] = 'Null'
        le = LabelEncoder()
        result = le.fit_transform(df.user_id.values).reshape(-1,1)
        return result

class SpaceFeatExt(object):
    
    def __init__(self):
        pass
    
    def extract_space(self, df):
        scaler = MinMaxScaler()
        result = scaler.fit_transform(df[['latitude','longitude']])
        return result
    
    def extract_space_dist(self, df, shop):
        def squaer_dist(loc1, loc2_list):
            dist = np.power(np.sum(np.power(loc2_list-loc1,2),1),0.5)
            return dist
        df_space = df[['latitude','longitude']].values
        shop_loc = shop[['latitude','longitude']].values
        feat = np.apply_along_axis(squaer_dist, 1, df_space, **{'loc2_list':shop_loc})
        return feat

class WifiFeatExt(object):
    
    def __init__(self):
        pass
    
    def fit(self, df, drop_rate=None, drop_num=None):
        self.wifi_dict = self.get_wifi_dict(df, drop_rate, drop_num)
    
    def extract_wifi_power(self, df):
        wifi_infos = df.wifi_infos
        result = []
        for wifi_infos_i in wifi_infos:
            wifi_infos_i = [wifi.split('|') for wifi in wifi_infos_i.split(';')]
            result_i = self.wifi_dict.copy()
            for wifi in wifi_infos_i:
                if wifi[0] in self.wifi_dict:
                    result_i[wifi[0]] = float(wifi[1]) + 120
            result.append(result_i)
        result = pd.DataFrame(result)
        return result
    
    def extract_wifi_connect(self, df):
        label_encoder = LabelEncoder()
        wifi_connect = []
        wifi_infos = df['wifi_infos']
        for wifi_infos_i in wifi_infos:
            wifi_infos_i = [wifi.split('|') for wifi in wifi_infos_i.split(';')]
            flag = True
            for wifi in wifi_infos_i:
                if wifi[2] != 'false':
                    wifi_connect.append(wifi[0])
                    flag = False
                    break
            if flag:
                wifi_connect.append('Null')
        wifi_connect = label_encoder.fit_transform(wifi_connect).reshape(-1,1)
        return wifi_connect
            
        
    def get_wifi_dict(self, df, drop_rate=None, drop_num=None):
        wifi_infos = df['wifi_infos']
        wifi_count = {}
        wifi_connect = []
        for wifi_infos_i in wifi_infos:
            wifi_infos_i = [wifi.split('|') for wifi in wifi_infos_i.split(';')]
            for wifi in wifi_infos_i:
                if wifi[0] in wifi_count:
                    wifi_count[wifi[0]] += 1
                else:
                    wifi_count[wifi[0]] = 1
        if drop_rate:
            n = np.percentile(pd.Series(wifi_count).values, drop_rate*100)
        if drop_num:
            n = drop_num
        wifi_dict = {}
        for k, v in wifi_count.items():
            if v > n:
                wifi_dict[k] = 0
        return wifi_dict

class TimeFeatExt(object):
    
    def __init__(self):
        pass
    
    def extract_weekday(self, df, onehot=True):
        result = df.time_stamp.dt.weekday.values.reshape(df.shape[0], 1)
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result
    
    def extract_hour(self, df, onehot=True):
        result = df.time_stamp.dt.hour.values.reshape(df.shape[0], 1)
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result
    
    def extract_hour_map(self, df, onehot=True):
        def time_map(t):
            if (t >= datetime.time(22,0)) or (t < datetime.time(8,0)):
                return 0
            if (t >= datetime.time(8,0)) and (t < datetime.time(9,30)):
                return 1
            if (t >= datetime.time(9,30)) and (t < datetime.time(11,30)):
                return 2
            if (t >= datetime.time(11,30)) and (t < datetime.time(14,0)):
                return 3
            if (t >= datetime.time(14,0)) and (t < datetime.time(17,0)):
                return 4
            if (t >= datetime.time(17,0)) and (t < datetime.time(20,0)):
                return 5
            if (t >= datetime.time(20,0)) and (t < datetime.time(22,0)):
                return 6
        result = df.time_stamp.dt.time.apply(time_map).values.reshape(df.shape[0], 1)
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result
    
    def extract_weekday_hour_map(self, df, onehot=True):
        weekday = self.extract_weekday(df, onehot=False)
        hour_map = self.extract_hour_map(df, onehot=False)
        result = weekday * 100 + hour_map
        if onehot:
            result = pd.get_dummies(pd.Series(result.reshape(-1))).values
        return result